# Demand Response potential comparison
**Purpose and background**:
* Compare technical demand response potentials in terms of positive and negative shifting potential with the literature.
* For this purpose, read in data extracted in notebook "DR_potential_evaluation" on the level of load shifting categories.

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from drpotentials.compare_potentials import plot_potential_comparison

## Workflow settings

In [ ]:
path_folder_stats = "./out/stats/"
sub_folder_individual_studies = "individual_studies/"

path_folder_inputs = "./inputs/"
file_name_colors = "Potenziale_Lastmanagement.xlsx"

sectors = ["ind", "tcs", "hoho"]
dr_scenarios = ["5%", "50%", "95%"]
params = [
    "potential_pos_overall", 
    "potential_neg_overall",
    "potential_pos_overall_shed",
]
years = ["SQ", "2020", "2030", "2040", "2050"]
studies = ["Ape12", "Gil15", "Hei21", "Jet21", "Klo09", "Lad18", "Ste17"]
cols_to_evaluate = ["5%", "50%", "95%"]

file_names_stats = {}
file_names_stats_studies = {}
for year in years:
    file_names_stats[year] = {}
    file_names_stats_studies[year] = {}
    for param in params:
        file_names_stats[year][param] = [
            f"{path_folder_stats}{param}_{sector}_stats_{year}.csv" 
            for sector in sectors
        ]
        file_names_stats_studies[year][param] = {}
        for study in studies:
            file_names_stats_studies[year][param][study] = [
                f"{path_folder_stats}{sub_folder_individual_studies}{param}_{sector}_stats_{study}_{year}.csv" 
                for sector in sectors
            ]

Configure the fontsizes for matplotlib objects

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Read in and preprocess data

In [ ]:
colors = pd.read_excel(f"{path_folder_inputs}{file_name_colors}", sheet_name="Farben", index_col=0).drop(columns="Prozesskategorie short")["Farbe (matplotlib strings)"]

In [ ]:
own_potentials = {}
for year in years:
    own_potentials[year] = {}
    for param in params:
        potentials = pd.DataFrame()
        for file in file_names_stats[year][param]:
            try:
                potential = pd.read_csv(file, sep=";", index_col=0).loc[cols_to_evaluate]
            except FileNotFoundError:
                continue
            potentials = pd.concat([potentials, potential], axis=1)

        # Add duplicate entries together (such as heat pumps or cross-cutting technologies)
        potentials = potentials.groupby(axis=1, level=0).sum()
        potentials["study"] = ["Eigene"] * len(potentials)
        potentials = potentials.set_index("study", append=True)
        own_potentials[year][param] = potentials

In [ ]:
all_studies_potentials = {}
for year in years:
    all_studies_potentials[year] = {}
    for param in params:
        to_concat = []
        for study in studies:
            potentials_study = pd.DataFrame()
            for file in file_names_stats_studies[year][param][study]:
                try:
                    potential = pd.read_csv(file, sep=";", index_col=0).loc[["max"]]
                except FileNotFoundError:
                    continue
                potentials_study = pd.concat([potentials_study, potential], axis=1)

            # Add duplicate entries together (such as heat pumps or cross-cutting technologies)
            potentials_study = potentials_study.groupby(axis=1, level=0).sum()
            potentials_study["study"] = [study] * len(potentials_study)
            potentials_study = potentials_study.set_index("study", append=True)
            to_concat.append(potentials_study)
        potentials_all_studies = pd.concat(to_concat)
        all_studies_potentials[year][param] = potentials_all_studies

In [ ]:
combined_potentials = {}
for year in years:
    combined_potentials[year] = {}
    for param in params:
        combined = pd.concat([own_potentials[year][param], all_studies_potentials[year][param]])
        combined.index = combined.index.get_level_values(0) + " " + combined.index.get_level_values(1)
        combined_potentials[year][param] = combined.rename(index={f"max {study}": study for study in studies})

## Create plots for comparing potentials

In [ ]:
for param in params:
    plot_potential_comparison(
        combined_potentials["SQ"][param],
        colors,
        figsize=(18, 10),
        log_scale=False,
        show=True,
        save=True,
        path="./out/plots/",
        file_name=f"potential_comparison_{param}",
    )